In [ ]:
import pandas as pd
import json
from os import path

In [ ]:
def compute_agreement_rate(csv_path, json_path):
    df = pd.read_csv(csv_path)
    
    df['what'] = df['what'].astype(bool)
    df['why'] = df['why'].astype(bool)

    with open(json_path, 'r') as f:
        llm_data = json.load(f)
    
    results = {}

    for llm_name, ratings in llm_data.items():
        matches = 0
        total = 0

        for commit in ratings:
            sha = commit["sha"]
            llm_what = bool(commit.get("what_present", False))
            llm_why = bool(commit.get("why_present", False))
            
            row = df[df['sha'] == sha]
            if row.empty:
                continue

            row = row.iloc[0]
            total += 2 

            if bool(row['what']) == llm_what:
                matches += 1
            if bool(row['why']) == llm_why:
                matches += 1

        results[llm_name] = matches / total if total > 0 else None

    return results

In [ ]:
rate_agreement = compute_agreement_rate(path.abspath(
    path.join('..', '..', 'data', 'chosen_commits.csv')),
    path.abspath(path.join('..', '..', 'evaluation_for_llama3_model.json')
))

llama3.1:8b: 31.25%


In [ ]:
rate_agreement_df.index.name = "model_name"
rate_agreement_df.to_csv(path.abspath(
    path.join('..', '..', 'evaluation_for_llama3_model_agreement_rate.csv')), index=True)

: 